In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import itertools


In [ ]:
# Dados originais
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]
media_2017 = [26.00, 26.00, 23.00, 23.00, 20.00, 18.00, 17.00, 21.00, 23.00, 25.00, 22.00, 25.00]
media_2018 = [24.18, 25.39, 26.13, 22.30, 19.18, 21.40, 18.51, 19.88, 21.87, 23.62, 24.00, 24,00]
media_2019 = [26.58, 26.0, 24.86, 23.5, 20.68, 19.23, 17.41, 21.14, 24.5, 25.73, 24.9, 24.9]
media_2020_parcial = [np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, 26.12, 26.13, 23.44, 25.15]
media_2021 = [25.41, 25.3, 24.19, 21.34, 18.99, 20.07, 20.2, 22.78, 24.95, 23.43, 23.51, 23.46]
media_2022 = [25.27, 24.99, 24.84, 21.84, 17.48, 18.4, 18.96, 20.46, 22.04, 24.4, 22.0, 24.56]
media_2023 = [24.91, 24.96, 25.14, 21.51, 19.49, 18.34, 19.41, 22.82, 24.54, 25.27, 26.02, 26.18]
media_2024 = [25.84, 25.46, 26.12, 24.05, 22.0, 21.04, 20.08, 19.9, 25.87, 26.42, 24.17, 25.16]
media_2025_parcial = [25.0, 26.08, 25.03, 23.04, 18.4, 16.05, 18.9, 19.1, np.nan, np.nan, np.nan, np.nan]

---
A obesercação de que o valor de 13.36°C para janeiro de 2020 é uma grande disparidade é um ponto crucial na análise de dados. Mesmo com mais dados e parâmetros otimizados.

Isso acontece porque, o modelo SARIMA é uma extrapolação **baseada em padrões**. Uma lacuna de 8 meses nos dados (de janeiro a agosto de 2020) é um período muito longo, e o modelo pode ter dificuldade em "pular" essa lacuna sem gerar um valor estranho no ponto de partida. Ele pode estar reagindo a um padrão de queda de temperatura no final de 2019 e extrapolando essa tendência de forma exagerada para o início de 2020.

Neste caso, em vez de usar uma previsão estatística para preencher uma lacuna dentro do histórico, vamos usar a alternativa mais robusta e simples, a **interpolação**. A interpolação preenche os valores ausentes de forma suave, "conectando" os pontos de dados conhecidos antes e depois da lacuna.

Como vemos os valores comparados entre a **extrapolação(previsão)** e a **interpolação** temos a solução para o primero mês de 2020, sendo mais coerente porem os demais proximos 7 meses tem valor ligeramente maiores com a interpolação comparado ao a extrapolação.

Nesse caso vamos usas  interpolação para preencher a lacuna de 2020 de forma suave e realista, evitando os valores extremos que um modelo poderia gerar e emseguida, use a série de dados agora completa e limpa para treinar o modelo SARIMA e fazer uma previsão mais precisa para o futuro (2025).

In [ ]:
# Ignorando avisos para uma saída mais limpa
warnings.filterwarnings("ignore")
# Combinando todos os dados em uma única lista
temperaturas_completas = media_2019 + media_2020_parcial + media_2021 + media_2022 + media_2023 + media_2024 + media_2025_parcial
datas_completas = pd.date_range(start='2017-01', periods=len(temperaturas_completas), freq='ME')
serie_temporal = pd.Series(temperaturas_completas, index=datas_completas)

# --- Otimização de Parâmetros (Grid Search) ---
p = d = q = range(0, 2)
P = D = Q = range(0, 2)
s = 12

pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], s) for x in list(itertools.product(P, D, Q))]

melhor_aic = float('inf')
melhores_parametros = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            modelo_sarima = SARIMAX(serie_temporal,
                                    order=param,
                                    seasonal_order=param_seasonal,
                                    enforce_stationarity=False,
                                    enforce_invertibility=False)
            resultado = modelo_sarima.fit(disp=False)

            if resultado.aic < melhor_aic:
                melhor_aic = resultado.aic
                melhores_parametros = (param, param_seasonal)
        except Exception as e:
            continue

print(f"Melhores parâmetros SARIMA encontrados: {melhores_parametros} com AIC de {melhor_aic:.2f}")

# --- Treinando o modelo com os melhores parâmetros ---
modelo_otimizado = SARIMAX(serie_temporal,
                           order=melhores_parametros[0],
                           seasonal_order=melhores_parametros[1],
                           enforce_stationarity=False,
                           enforce_invertibility=False)
resultado_otimizado = modelo_otimizado.fit(disp=False)

# --- Gerando as novas previsões ---
previsao_2020_df = resultado_otimizado.get_prediction(start='2020-01-31', end='2020-08-31')
previsao_2020 = previsao_2020_df.predicted_mean.round(2).tolist()

previsao_2025 = resultado_otimizado.forecast(steps=4).round(2).tolist()

# --- Exibindo os novos resultados ---
print("\nNovos valores imputados para os primeiros 8 meses de 2020:")
print(previsao_2020)
print("\nNovos valores previstos para os últimos 4 meses de 2025:")
print(previsao_2025)

In [ ]:
# Mapeando os dados para cada ano
dados_temperatura = {
    '2019': media_2019,
    '2020': media_2020,
    '2021': media_2021,
    '2022': media_2022,
    '2023': media_2023,
    '2024': media_2024,
    '2025': media_2025,
}

# Criando o DataFrame
df_temperaturas = pd.DataFrame(dados_temperatura)

# Definindo os meses como o índice das linhas
df_temperaturas.index = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Exibindo a tabela
df_temperaturas.head(12)



In [ ]:

# Dados do seu gráfico
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
precipitacao = [34.175, 23.425, 19.5, 7.675, 7.45, 5.1625, 4.7875, 3.9, 7.55, 14.4875, 20.15, 25.7125]
temperatura_max = [33.9, 33.8, 33.0, 32.0, 30.0, 28.6, 29.5, 32.2, 34.5, 35.0, 34.1, 33.8]
temperatura_min = [16.5, 16.9, 16.3, 12.7, 9.0, 8.0, 7.1, 8.7, 10.8, 13.3, 13.9, 15.7]

# --- Configuração do Gráfico ---
fig, ax1 = plt.subplots(figsize=(12, 8))

# Título e Legenda do seu código original
plt.title('Precipitação/Temperatura - 1990/2024', fontsize=16, pad=20)
description = 'Média mensal da temperatura máxima (em graus Celsius °C), da temperatura mínima (em graus Celsius °C) e da precipitação (em milímetros), dos dados registrados na estação meteorológica do CEPAGRI/Unicamp, no período de 1990 a 2024'
plt.figtext(0.5, 0.01, description, wrap=True, horizontalalignment='center', fontsize=10)

# Gráfico de barras para a precipitação
ax1.bar(meses, precipitacao, color='#00CCFF', label='Precipitação')
ax1.set_xlabel('Meses')
ax1.set_ylabel('Precipitação (mm)')
ax1.tick_params(axis='y')
ax1.set_ylim([0, max(precipitacao) * 1.5])

# Cria um segundo eixo Y para as temperaturas
ax2 = ax1.twinx()
ax2.plot(meses, temperatura_max, color='#DD0000', label='Máxima', marker='o')
ax2.plot(meses, temperatura_min, color='#0033DD', label='Mínima', marker='o')
ax2.set_ylabel('Temperatura (°C)')
ax2.tick_params(axis='y')
ax2.set_ylim([0, max(temperatura_max) + 5])

# Configurações adicionais
fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0.95), ncol=3)
fig.tight_layout(rect=[0, 0.05, 1, 0.9])
plt.grid(True, linestyle='--', alpha=0.6)

# Exibir o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Seus dados de temperatura
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
temperatura_maxima = [33.9, 33.8, 33.0, 32.0, 30.0, 28.6, 29.5, 32.2, 34.5, 35.0, 34.1, 33.8]
temperatura_media = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]
temperatura_minima = [16.5, 16.9, 16.3, 12.7, 9.0, 8.0, 7.1, 8.7, 10.8, 13.3, 13.9, 15.7]

# Configuração do gráfico
plt.figure(figsize=(10, 6))

# Adicionando as linhas para as temperaturas
plt.plot(meses, temperatura_maxima, marker='o', label='Máxima', color='#e2431e')
plt.plot(meses, temperatura_media, marker='o', label='Média', color='#00DD66')
plt.plot(meses, temperatura_minima, marker='o', label='Mínima', color='#0066DD')

# Adicionando título e rótulos
plt.title('Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=3)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da Temperatura máxima, temperatura média e temperatura mínima, em graus Celsius (°C), dos dados registrados na Estação Meteorológica do Cepagri/Unicamp, no período de 1990 a 2024',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)
media_2019 = [26.58, 26.0, 24.86, 23.5, 20.68, 19.23, 17.41, 21.14, 24.5, 25.73, 24.9, 24.9]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2019, marker='o', label='Média 2019', color='#00DD65')


# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

# Ignorando avisos para uma saída mais limpa
warnings.filterwarnings("ignore")

# Combinando todos os dados de temperatura em uma lista
todas_as_temperaturas = media_2019 + media_2020 + media_2021 + media_2022 + media_2023 + media_2024 + media_2025

# Criando a série temporal, excluindo os zeros no final
datas = pd.date_range(start='2019-01', periods=len(todas_as_temperaturas), freq='M')
serie_temporal = pd.Series(todas_as_temperaturas, index=datas)
serie_temporal_validos = serie_temporal[serie_temporal > 0]

# Definindo o modelo SARIMA
# Parâmetros:
# order=(1, 1, 1): Ordem padrão do ARIMA
# seasonal_order=(0, 1, 1, 12): Ordem sazonal, com período de 12 meses (para o ciclo anual)
modelo_sarima = SARIMAX(serie_temporal_validos,
                      order=(1, 1, 1),
                      seasonal_order=(0, 1, 1, 12),
                      enforce_stationarity=False,
                      enforce_invertibility=False)

# Treinando o modelo
resultado = modelo_sarima.fit(disp=False)

# Previsão para os próximos 4 meses
previsao_sarima = resultado.forecast(steps=4)

print("Previsão das temperaturas para Setembro, Outubro, Novembro e Dezembro de 2025 (com SARIMA):")
print(previsao_sarima)

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)
media_2020 = [0, 0, 0, 0, 0, 0, 0, 0, 26.12, 26.13, 23.44, 25.15]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2020, marker='o', label='Média 2020', color='#00DD65')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)

media_2021 = [25.41, 25.30, 24.19, 21.34, 18.99, 20.07, 20.20, 22.78, 24.95, 23.43, 23.51, 23.46]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2021, marker='o', label='Média 2021', color='#00DD65')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)

media_2022 = [25.27, 24.99, 24.84, 21.84, 17.48, 18.4, 18.96, 20.46, 22.04, 24.4, 22.0, 24.56]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2022, marker='o', label='Média 2022', color='#00DD65')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)

media_2023 = [24.91, 24.96, 25.14, 21.51, 19.49, 18.34, 19.41, 22.82, 24.54, 25.27, 26.02, 26.18]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2023, marker='o', label='Média 2023', color='#00DD65')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)

media_2024 = [25.84, 25.46, 26.12, 24.05, 22.00, 21.04, 20.08, 19.90, 25.87, 26.42, 24.17, 25.16]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2024, marker='o', label='Média 2024', color='#00DD65')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
plt.plot(meses, media_2025, marker='o', label='Média 2025', color='#00DD65')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)

media_2025 = [25.00, 26.08, 25.03, 23.04, 18.40, 16.05, 18.90, 19.1, 0, 0, 0, 0]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
#plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
#plt.plot(meses, media_2019, marker='o', label='Média 2019', color='#cd7f32')
#plt.plot(meses, media_2020, marker='o', label='Média 2020', color='#00DD65')
#plt.plot(meses, media_2021, marker='o', label='Média 2021', color='#00DD65')
plt.plot(meses, media_2022, marker='o', label='Média 2022', color='#d4be2b')
plt.plot(meses, media_2023, marker='o', label='Média 2023', color='#00ffe6')
plt.plot(meses, media_2024, marker='o', label='Média 2024', color='#00DD65')
plt.plot(meses, media_2025, marker='o', label='Média 2025', color='#e0371f')

# Adicionando título e rótulos
plt.title('Comparação/Temperatura - 1990/2024')
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, em comparação à média de temperatura média registrada no período de 1990/2024, da Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()

In [ ]:
# Mapeando os dados para cada ano
dados_temperatura = {
    '2019': media_2019,
    '2020': media_2020,
    '2021': media_2021,
    '2022': media_2022,
    '2023': media_2023,
    '2024': media_2024,
    '2025': media_2025,
}

# Criando o DataFrame
df_temperaturas = pd.DataFrame(dados_temperatura)

# Definindo os meses como o índice das linhas
df_temperaturas.index = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']

# Exibindo a tabela
df_temperaturas.head(12)



In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings

# Ignorando avisos para uma saída mais limpa
warnings.filterwarnings("ignore")

# Combinando todos os dados de temperatura em uma lista
todas_as_temperaturas = media_2019 + media_2020 + media_2021 + media_2022 + media_2023 + media_2024 + media_2025

# Criando a série temporal, excluindo os zeros no final
datas = pd.date_range(start='2019-01', periods=len(todas_as_temperaturas), freq='M')
serie_temporal = pd.Series(todas_as_temperaturas, index=datas)
serie_temporal_validos = serie_temporal[serie_temporal > 0]

# Definindo o modelo SARIMA
# Parâmetros:
# order=(1, 1, 1): Ordem padrão do ARIMA
# seasonal_order=(0, 1, 1, 12): Ordem sazonal, com período de 12 meses (para o ciclo anual)
modelo_sarima = SARIMAX(serie_temporal_validos,
                      order=(1, 1, 1),
                      seasonal_order=(0, 1, 1, 12),
                      enforce_stationarity=False,
                      enforce_invertibility=False)

# Treinando o modelo
resultado = modelo_sarima.fit(disp=False)

# Previsão para os próximos 4 meses
previsao_sarima = resultado.forecast(steps=4)

print("Previsão das temperaturas para Setembro, Outubro, Novembro e Dezembro de 2025 (com SARIMA):")
print(previsao_sarima)

In [ ]:
media_2025_previsao = [25.00, 26.08, 25.03, 23.04, 18.40, 16.05, 18.90, 19.1, 23.38, 24.16, 22.98, 23.88]

In [ ]:
# Dados históricos (média de 1990-2024)
meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
media_historica = [25.2, 25.3, 24.7, 22.3, 19.4, 18.2, 18.3, 20.4, 22.7, 24.1, 24.0, 24.9]

# Dados fictícios para um ano específico (exemplo: 2024)

media_2025 = [25.00, 26.08, 25.03, 23.04, 18.40, 16.05, 18.90, 19.1, 0, 0, 0, 0]

# Configuração do gráfico
plt.figure(figsize=(12, 7))

# Plotando as linhas de temperatura
#plt.plot(meses, media_historica, marker='o', label='Média Histórica (1990-2024)', color='#e2431e')
#plt.plot(meses, media_2019, marker='o', label='Média 2019', color='#cd7f32')
#plt.plot(meses, media_2020, marker='o', label='Média 2020', color='#00DD65')
#plt.plot(meses, media_2021, marker='o', label='Média 2021', color='#00DD65')
#plt.plot(meses, media_2022, marker='o', label='Média 2022', color='#d4be2b')
plt.plot(meses, media_2023, marker='o', label='Média 2023', color='#00ffe6')
plt.plot(meses, media_2024, marker='o', label='Média 2024', color='#00DD65')
# Plotando a linha de 2025 em dois segmentos
# 1. Dados conhecidos (até agosto): linha sólida
plt.plot(meses[:8], media_2025_previsao[:8], marker='o', color='#e0371f', label='Média 2025')
# 2. Previsão (a partir de setembro): linha tracejada
plt.plot(meses[7:], media_2025_previsao[7:], marker='o', color='#e0371f', label='Previsão Media 2025', linestyle='--')

# Adicionando título e rótulos
plt.xlabel('Mês')
plt.ylabel('Temperatura (°C)')
plt.grid(True, linestyle='--', alpha=0.6)

# Adicionando a legenda do gráfico
plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=2)

# Adicionando a descrição do gráfico
plt.figtext(0.5, 0.01, 
            'Média mensal da temperatura média, em graus Celsius, com base na Estação Meteorológica do Cepagri/Unicamp',
            wrap=True, horizontalalignment='center', fontsize=10)

# Define o limite do eixo Y para começar em 0 e terminar em 40
plt.ylim(0, 40) 

# Ajustando o layout para evitar sobreposição
plt.tight_layout(rect=[0, 0.05, 1, 0.95])

# Exibindo o gráfico
plt.show()